# Load packages and dependencies

In [2]:
import os
import json
import math
# from google.colab import drive

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow.keras as keras

import librosa
import IPython.display as display

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

2024-01-13 12:08:48.737739: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-13 12:08:48.817739: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-13 12:08:48.817799: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-13 12:08:48.819675: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-13 12:08:48.830942: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-13 12:08:48.831828: I tensorflow/core/platform/cpu_feature_guard.cc:1

# Connect the drive

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


# Load the raw data

In [4]:
DATASET_PATH = 'dataset"
JSON_PATH = "data.json"
SAMPLE_RATE = 16000
AUDIO_DURATION = 2  # measured in seconds
SAMPLES_PER_AUDIO = SAMPLE_RATE * AUDIO_DURATION

SyntaxError: unterminated string literal (detected at line 1) (1185739606.py, line 1)

# Preprocess the data

In [4]:
def save_mfcc(dataset_path, json_path, num_mfcc=13, n_fft=2048, hop_length=512, num_segments=5):

    # dictionary to store mapping, labels, and MFCCs
    data = {
        "mapping": [],
        "labels": [],
        "mfcc": []
    }

    samples_per_segment = int(SAMPLES_PER_AUDIO / num_segments)
    num_mfcc_vectors_per_segment = math.ceil(samples_per_segment / hop_length)

    # loop through all sub-folders
    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):

        # ensure we're processing the sub-folder level
        if dirpath is not dataset_path:

            # save genre label (i.e., sub-folder name) in the mapping
            semantic_label = dirpath.split("/")[-1]
            data["mapping"].append(semantic_label)
            print(f"\nProcessing: {semantic_label}")

            # process all audio files in genre sub-dir
            for f in filenames:

                # load audio file
                file_path = os.path.join(dirpath, f)
                signal, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE)

                # process all segments of audio file
                for d in range(num_segments):

                    # calculate start and finish sample for the current segment
                    start = samples_per_segment * d
                    finish = start + samples_per_segment

                    # extract mfcc
                    mfcc = librosa.feature.mfcc(y=signal[start:finish],
                                                sr=sample_rate,
                                                n_mfcc=num_mfcc,
                                                n_fft=n_fft,
                                                hop_length=hop_length)

                    mfcc = mfcc.T

                    # store only mfcc feature with the expected number of vectors
                    if len(mfcc) == num_mfcc_vectors_per_segment:
                        data["mfcc"].append(mfcc.tolist())
                        data["labels"].append(i-1)
                        print(f"{file_path}, segment:{d+1}")

    # save MFCCs to json file
    with open(json_path, "w") as fp:
        json.dump(data, fp, indent=4)

    # return the data dictionary
    return data


## Save the mfcc data

In [ ]:
data = save_mfcc(DATASET_PATH, JSON_PATH, num_segments=6)


Processing: forward
/content/drive/My Drive/dataset/english/forward/9a76f8c3_nohash_3.wav, segment:1
/content/drive/My Drive/dataset/english/forward/9a76f8c3_nohash_3.wav, segment:2
/content/drive/My Drive/dataset/english/forward/9a76f8c3_nohash_3.wav, segment:3


/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(


/content/drive/My Drive/dataset/english/forward/3108acca_nohash_1.wav, segment:1
/content/drive/My Drive/dataset/english/forward/3108acca_nohash_1.wav, segment:2
/content/drive/My Drive/dataset/english/forward/3108acca_nohash_1.wav, segment:3
/content/drive/My Drive/dataset/english/forward/211b928a_nohash_0.wav, segment:1
/content/drive/My Drive/dataset/english/forward/211b928a_nohash_0.wav, segment:2
/content/drive/My Drive/dataset/english/forward/211b928a_nohash_0.wav, segment:3
/content/drive/My Drive/dataset/english/forward/85851131_nohash_1.wav, segment:1
/content/drive/My Drive/dataset/english/forward/85851131_nohash_1.wav, segment:2
/content/drive/My Drive/dataset/english/forward/85851131_nohash_1.wav, segment:3
/content/drive/My Drive/dataset/english/forward/40738a2d_nohash_3.wav, segment:1
/content/drive/My Drive/dataset/english/forward/40738a2d_nohash_3.wav, segment:2
/content/drive/My Drive/dataset/english/forward/40738a2d_nohash_3.wav, segment:3
/content/drive/My Drive/data

/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1622
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=939
  warnings.warn(


/content/drive/My Drive/dataset/english/forward/79903546_nohash_0.wav, segment:1
/content/drive/My Drive/dataset/english/forward/79903546_nohash_0.wav, segment:2
/content/drive/My Drive/dataset/english/forward/24cab2cb_nohash_1.wav, segment:1
/content/drive/My Drive/dataset/english/forward/24cab2cb_nohash_1.wav, segment:2
/content/drive/My Drive/dataset/english/forward/30060aba_nohash_3.wav, segment:1
/content/drive/My Drive/dataset/english/forward/30060aba_nohash_3.wav, segment:2
/content/drive/My Drive/dataset/english/forward/30060aba_nohash_3.wav, segment:3
/content/drive/My Drive/dataset/english/forward/8fe67225_nohash_4.wav, segment:1
/content/drive/My Drive/dataset/english/forward/8fe67225_nohash_4.wav, segment:2
/content/drive/My Drive/dataset/english/forward/8fe67225_nohash_4.wav, segment:3
/content/drive/My Drive/dataset/english/forward/0cb74144_nohash_0.wav, segment:1
/content/drive/My Drive/dataset/english/forward/0cb74144_nohash_0.wav, segment:2
/content/drive/My Drive/data

...


/content/drive/My Drive/dataset/english/stop/4c6944d6_nohash_2.wav, segment:3
/content/drive/My Drive/dataset/english/stop/0e6e36c9_nohash_4.wav, segment:1
/content/drive/My Drive/dataset/english/stop/0e6e36c9_nohash_4.wav, segment:2
/content/drive/My Drive/dataset/english/stop/0e6e36c9_nohash_4.wav, segment:3
/content/drive/My Drive/dataset/english/stop/1b88bf70_nohash_1.wav, segment:1
/content/drive/My Drive/dataset/english/stop/1b88bf70_nohash_1.wav, segment:2
/content/drive/My Drive/dataset/english/stop/1b88bf70_nohash_1.wav, segment:3
/content/drive/My Drive/dataset/english/stop/1b42b551_nohash_4.wav, segment:1
/content/drive/My Drive/dataset/english/stop/1b42b551_nohash_4.wav, segment:2
/content/drive/My Drive/dataset/english/stop/1b42b551_nohash_4.wav, segment:3
/content/drive/My Drive/dataset/english/stop/2b42e7a2_nohash_0.wav, segment:1
/content/drive/My Drive/dataset/english/stop/2b42e7a2_nohash_0.wav, segment:2
/content/drive/My Drive/dataset/english/stop/2b42e7a2_nohash_0.

/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=811
  warnings.warn(


/content/drive/My Drive/dataset/english/stop/9a69672b_nohash_2.wav, segment:2
/content/drive/My Drive/dataset/english/stop/9a69672b_nohash_2.wav, segment:3
/content/drive/My Drive/dataset/english/stop/1dc86f91_nohash_0.wav, segment:1
/content/drive/My Drive/dataset/english/stop/1dc86f91_nohash_0.wav, segment:2
/content/drive/My Drive/dataset/english/stop/1dc86f91_nohash_0.wav, segment:3
/content/drive/My Drive/dataset/english/stop/7fa5fa1f_nohash_0.wav, segment:1
/content/drive/My Drive/dataset/english/stop/7fa5fa1f_nohash_0.wav, segment:2
/content/drive/My Drive/dataset/english/stop/7fa5fa1f_nohash_0.wav, segment:3
/content/drive/My Drive/dataset/english/stop/1a994c9f_nohash_2.wav, segment:1
/content/drive/My Drive/dataset/english/stop/1a994c9f_nohash_2.wav, segment:2
/content/drive/My Drive/dataset/english/stop/1a994c9f_nohash_2.wav, segment:3
/content/drive/My Drive/dataset/english/stop/7c83c5f6_nohash_2.wav, segment:1
/content/drive/My Drive/dataset/english/stop/7c83c5f6_nohash_2.w

/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1835
  warnings.warn(


Streaming output truncated to the last 5000 lines.
/content/drive/My Drive/dataset/english/backward/235b444f_nohash_3.wav, segment:1
/content/drive/My Drive/dataset/english/backward/235b444f_nohash_3.wav, segment:2
/content/drive/My Drive/dataset/english/backward/235b444f_nohash_3.wav, segment:3
/content/drive/My Drive/dataset/english/backward/978240e1_nohash_0.wav, segment:1
/content/drive/My Drive/dataset/english/backward/978240e1_nohash_0.wav, segment:2
/content/drive/My Drive/dataset/english/backward/978240e1_nohash_0.wav, segment:3
/content/drive/My Drive/dataset/english/backward/79903546_nohash_1.wav, segment:1
/content/drive/My Drive/dataset/english/backward/79903546_nohash_1.wav, segment:2
/content/drive/My Drive/dataset/english/backward/79903546_nohash_1.wav, segment:3
/content/drive/My Drive/dataset/english/backward/4c6167ca_nohash_1.wav, segment:1
/content/drive/My Drive/dataset/english/backward/4c6167ca_nohash_1.wav, segment:2
/content/drive/My Drive/dataset/english/backwar

## Load json file

In [ ]:
def load_data(data):

    # convert lists to numpy arrays
    X = np.array(data["mfcc"])
    y = np.array(data["labels"])

    print("Data successfully loaded!")

    return X, y


In [ ]:
X, y = load_data(data)

Data successfully loaded!


# Split Data into Training and Testing Sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Build a Neural Network Model

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(X.shape[1], X.shape[2])),
    keras.layers.Dense(512, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(10, activation='softmax')
])

## Compile the model

In [ ]:
optimiser = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimiser,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 143)               0         
                                                                 
 dense (Dense)               (None, 512)               73728     
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 64)                16448     
                                                                 
 dropout_2 (Dropout)         (None, 64)                0

# Train the Model

In [ ]:
history = model.fit(X_train,
                    y_train,
                    validation_data=(X_test, y_test),
                    batch_size=32,
                    epochs=50)

## History

In [ ]:
def plot_history(history):

    fig, axs = plt.subplots(2)

    # create accuracy sublpot
    axs[0].plot(history.history["accuracy"], label="train accuracy")
    axs[0].plot(history.history["val_accuracy"], label="test accuracy")
    axs[0].set_ylabel("Accuracy")
    axs[0].legend(loc="lower right")
    axs[0].set_title("Accuracy eval")

    # create error sublpot
    axs[1].plot(history.history["loss"], label="train error")
    axs[1].plot(history.history["val_loss"], label="test error")
    axs[1].set_ylabel("Error")
    axs[1].set_xlabel("Epoch")
    axs[1].legend(loc="upper right")
    axs[1].set_title("Error eval")

    plt.show()

## Plot history

In [ ]:
plot_history(history)

# Evaluate the Model

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'\nTest accuracy: {test_acc}')
print(f'\nTest loss: {test_loss}')

# Make Predictions

In [ ]:
predictions = model.predict(X_test)

# Post-Processing and Analysis

# Save and Export Model

In [ ]:
model.save('audio_model.h5')

# confusion matrix

In [ ]:
y_pred = tf.argmax(y_pred, axis=1)

In [ ]:
y_true = tf.concat(list(test_spectrogram_ds.map(lambda s,lab: lab)), axis=0)

In [ ]:
confusion_mtx = tf.math.confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_mtx,
            xticklabels=label_names,
            yticklabels=label_names,
            annot=True, fmt='g')
plt.xlabel('Prediction')
plt.ylabel('Label')
plt.show()